<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Desafio 3: Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

## Imports y preparación dataset

In [32]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import pad_sequences
from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential
from tensorflow.python.client import device_lib

import glob

In [29]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True

In [2]:
read_files = glob.glob("corpus/*.txt")

with open("corpus/full_article.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

In [20]:
corpus_file = open("corpus/full_article.txt", "rb")
full_article = corpus_file.read()
display(f"Full article len: {len(full_article)}")
display(f"{full_article[4582:5015].decode("utf-8")}")

'Full article len: 4172723'

'But in the days of Bilbo, and of Frodo his heir, they suddenly \nbecame, by no wish of their own, both important and renowned, and troubled \n\n\n\n\nthe counsels of the Wise and the Great. \n\nThose days, the Third Age of Middle -earth, are now long past, and the \nshape of all lands has been changed; but the regions in which Hobbits then \nlived were doubtless the same as those in which they still linger: the \nNorth-West of the Old World'

In [26]:
max_context_size = 100
chars_vocab = set(full_article)
vocab_size = len(chars_vocab)
display(f"Vocabulary len: {vocab_size}")

'Vocabulary len: 105'

In [22]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

### Proceso de Tokenizar

In [24]:
# Vamos a "tolkienizar" el texto completo
tokenized_text = [char2idx[ch] for ch in full_article]
display(tokenized_text[4582:5015])

[29,
 79,
 78,
 1,
 67,
 72,
 1,
 78,
 66,
 63,
 1,
 62,
 59,
 83,
 77,
 1,
 73,
 64,
 1,
 29,
 67,
 70,
 60,
 73,
 10,
 1,
 59,
 72,
 62,
 1,
 73,
 64,
 1,
 33,
 76,
 73,
 62,
 73,
 1,
 66,
 67,
 77,
 1,
 66,
 63,
 67,
 76,
 10,
 1,
 78,
 66,
 63,
 83,
 1,
 77,
 79,
 62,
 62,
 63,
 72,
 70,
 83,
 1,
 0,
 60,
 63,
 61,
 59,
 71,
 63,
 10,
 1,
 60,
 83,
 1,
 72,
 73,
 1,
 81,
 67,
 77,
 66,
 1,
 73,
 64,
 1,
 78,
 66,
 63,
 67,
 76,
 1,
 73,
 81,
 72,
 10,
 1,
 60,
 73,
 78,
 66,
 1,
 67,
 71,
 74,
 73,
 76,
 78,
 59,
 72,
 78,
 1,
 59,
 72,
 62,
 1,
 76,
 63,
 72,
 73,
 81,
 72,
 63,
 62,
 10,
 1,
 59,
 72,
 62,
 1,
 78,
 76,
 73,
 79,
 60,
 70,
 63,
 62,
 1,
 0,
 0,
 0,
 0,
 0,
 78,
 66,
 63,
 1,
 61,
 73,
 79,
 72,
 77,
 63,
 70,
 77,
 1,
 73,
 64,
 1,
 78,
 66,
 63,
 1,
 50,
 67,
 77,
 63,
 1,
 59,
 72,
 62,
 1,
 78,
 66,
 63,
 1,
 34,
 76,
 63,
 59,
 78,
 12,
 1,
 0,
 0,
 47,
 66,
 73,
 77,
 63,
 1,
 62,
 59,
 83,
 77,
 10,
 1,
 78,
 66,
 63,
 1,
 47,
 66,
 67,
 76,
 62,
 1,
 28,
 

### Preparación de los dataset

In [27]:
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]
tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])
display(X.shape)
display(y.shape)

(3755323, 100)

(3755323, 100)

## Modelos 

### SimpleRNN

In [33]:
model = Sequential()
model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(300, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
display(device_lib.list_local_devices())
model.summary()

c:\Users\Pablo\IA-repos\Desafios_NLP\venv\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3226435578245359148
 xla_global_id: -1]

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_1              │ (None, None, 105)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, None, 300)      │       121,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 105)      │        31,605 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 153,405 (599.24 KB)

 Trainable params: 153,405 (599.24 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
history_ppl = []
hist = model.fit(X, y, epochs=20, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=512)
epoch_count = range(1, len(history_ppl) + 1)
sns.lineplot(x=epoch_count,  y=history_ppl)
plt.show()

Epoch 1/20
 693/7335 ━━━━━━━━━━━━━━━━━━━━ 24:38 223ms/step - loss: 2.7211

KeyboardInterrupt: 